In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load Dataset

In [ ]:
variableData = pd.read_csv('Xente_Variable_Definitions.csv')
data = pd.read_csv('data.csv')

In [ ]:
variableData

,Column Name,Definition
0,TransactionId,Unique �transaction identifier on platform
1,BatchId,Unique number assigned to a batch of transacti...
2,AccountId,Unique number identifying the customer on plat...
3,SubscriptionId,Unique number identifying the customer subscri...
4,CustomerId,Unique identifier attached to Account
5,CurrencyCode,Country currency
6,CountryCode,Numerical geographical code of country
7,ProviderId,Source provider of Item �bought.
8,ProductId,Item name being bought.
9,ProductCategory,ProductIds are organized into these broader pr...


In [ ]:
data.sample(5)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
40906,TransactionId_97010,BatchId_134460,AccountId_3863,SubscriptionId_3514,CustomerId_4309,UGX,256.0,ProviderId_5,ProductId_1,airtime,ChannelId_3,5000.0,5000.0,2018-12-26T11:36:26Z,4.0,0.0
27372,TransactionId_17823,BatchId_130671,AccountId_4841,SubscriptionId_3829,CustomerId_448,UGX,256.0,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50.0,2018-12-14T17:40:47Z,2.0,0.0
18380,TransactionId_43145,BatchId_80617,AccountId_4841,SubscriptionId_3829,CustomerId_2324,UGX,256.0,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50.0,2018-12-06T16:33:22Z,2.0,0.0
60475,TransactionId_132809,BatchId_13813,AccountId_3061,SubscriptionId_475,CustomerId_3487,UGX,256.0,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000.0,2019-01-14T20:05:39Z,2.0,0.0
44306,TransactionId_136214,BatchId_106726,AccountId_4841,SubscriptionId_3829,CustomerId_343,UGX,256.0,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-5000.0,5000.0,2018-12-28T12:58:53Z,2.0,0.0


# EDA

# Data Overview and Summary

In [ ]:
def summarize_data(df):
    # Basic Information
    def basic_info(df):
        print(df.info())
        print(df.describe(include='all'))
        print(df.isnull().sum())

    # Specific Summarizations
    def specific_summarizations(df):
        num_records = df.shape[0]
        num_unique_values = df.nunique()

        print(f'Number of records: {num_records}')
        print('Number of unique values per column:')
        print(num_unique_values)

        for column in df.columns:
            if df[column].dtype == 'object':
                print(f'\nTop 5 most frequent values in column "{column}":')
                print(df[column].value_counts().head())
            elif df[column].dtype in ['int64', 'float64']:
                total = df[column].sum()
                mean = df[column].mean()
                print(f'\nTotal of column "{column}": {total}')
                print(f'Mean of column "{column}": {mean:.2f}')

                if pd.to_datetime(df[column], errors='coerce').notnull().all():
                    df[column] = pd.to_datetime(df[column])
                    messages_per_period = df[column].dt.to_period('M').value_counts().sort_index()
                    print(f'\nNumber of records per month for column "{column}":')
                    print(messages_per_period)

        numeric_columns = df.select_dtypes(include=['number']).columns
        if not numeric_columns.empty:
            top_messages = df.nlargest(5, numeric_columns[0])
            print('\nTop 5 records by the first numerical column:')
            print(top_messages)
        else:
            print('\nNo numerical columns to display top records.')


    # Calling functions
    print("Basic Information:")
    basic_info(df)
    print("\nSpecific Summarizations:")
    specific_summarizations(df)

In [ ]:
summarize_data(data)

Basic Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70535 entries, 0 to 70534
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TransactionId         70535 non-null  object 
 1   BatchId               70534 non-null  object 
 2   AccountId             70534 non-null  object 
 3   SubscriptionId        70534 non-null  object 
 4   CustomerId            70534 non-null  object 
 5   CurrencyCode          70534 non-null  object 
 6   CountryCode           70534 non-null  float64
 7   ProviderId            70534 non-null  object 
 8   ProductId             70534 non-null  object 
 9   ProductCategory       70534 non-null  object 
 10  ChannelId             70534 non-null  object 
 11  Amount                70534 non-null  float64
 12  Value                 70534 non-null  float64
 13  TransactionStartTime  70534 non-null  object 
 14  PricingStrategy       70534 non-null  float64
 15  

# Droping Id missing values because we can't impute Id since it should be unique

In [ ]:
data.dropna(subset=['BatchId','CustomerId', 'AccountId', 'SubscriptionId','ProviderId', 'ProductId', 'ChannelId' ], inplace=True)

In [ ]:
data.isnull().sum()

TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
FraudResult             0
dtype: int64

In [ ]:
data.nunique()

TransactionId           70534
BatchId                 69841
AccountId                2954
SubscriptionId           2951
CustomerId               3045
CurrencyCode                1
CountryCode                 1
ProviderId                  6
ProductId                  23
ProductCategory             9
ChannelId                   4
Amount                   1312
Value                    1202
TransactionStartTime    69723
PricingStrategy             4
FraudResult                 2
dtype: int64

# Univariant Analysis of Numerical and Catagorical Data

In [ ]:
def univariate_analysis(df):
    def analyze_column(column):
        print(f"Analysis for column: {column.name}")
        # Visualization
        plt.figure(figsize=(10, 6))

        if pd.api.types.is_numeric_dtype(column):
            sns.histplot(column, kde=True, bins=30)
            plt.title(f'Histogram of {column.name}')
            plt.xlabel(column.name)
            plt.ylabel('Frequency')
        elif pd.api.types.is_datetime64_any_dtype(column):
            column.value_counts().sort_index().plot(kind='line')
            plt.title(f'Time Series of {column.name}')
            plt.xlabel('Date')
            plt.ylabel('Frequency')
        else:
            column.value_counts().plot(kind='bar')
            plt.title(f'Bar Plot of {column.name}')
            plt.xlabel(column.name)
            plt.ylabel('Frequency')

        plt.show()
        print("\n\n")

    for column in df.columns:
        analyze_column(df[column])

# Correlation Analysis

In [ ]:
catagorical_columns = ['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'PricingStrategy','FraudResult']
selected_columns = ['Amount', 'Value', 'TransactionStartTime','FraudResult']
# Perform correlation analysis on selected columns
correlation_matrix = data[selected_columns].corr()

# Print correlation matrix
print("Correlation Matrix:")
print(correlation_matrix)

# Outlier Detection

In [ ]:
def detect_outliers(df):
    # Iterate over each numeric column
    for col in df.select_dtypes(include='number'):
        # Plotting boxplot
        plt.figure(figsize=(10, 6))
        plt.boxplot(df[col], vert=False)
        plt.xlabel(col)
        plt.title(f'Boxplot for {col}')
        plt.show()

        # Detecting outliers using IQR
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        print(f"Outliers detected in {col}:")
        print(outliers)
        print('\n')

In [ ]:
detect_outliers(data)